In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import talib
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch.optim as optim
import os
from sklearn.model_selection import TimeSeriesSplit

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import seaborn as sns

import optuna
from optuna.samplers import TPESampler
from optuna.trial import TrialState
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau 
from optuna.pruners import HyperbandPruner
import shap
import plotly.graph_objs as go
import plotly.offline as pyo
from tqdm.auto import tqdm
from sklearn.utils.class_weight import compute_class_weight
import torch.nn.functional as F
from tqdm.notebook import tqdm
from IPython.display import display


from sklearn.model_selection import KFold
import copy
from torch.cuda.amp import GradScaler, autocast
import os


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("gpu")
else:
    device = torch.device('cpu')
print(torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
print('cuDNN version:', torch.backends.cudnn.version())


2.1.2+cu121
CUDA available: False
CUDA version: 12.1
cuDNN version: 8902


/home/arda/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning:

CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)



In [3]:
pivoted_data = pd.read_csv("../Data/filled_data_long_pivot.csv")
# pivoted_data.drop(columns=["Unnamed: 0"], inplace=True)
#pivoted_data.set_index("PENDS", inplace=True)
# pivoted_data = pivoted_data.rename(columns={"Report Date" : "PENDS" ,"Ticker":"OFTIC"})
len(pivoted_data["OFTIC"].unique())
# pivoted_data[pivoted_data["OFTIC"] == "AAPL"]
pivoted_data = pivoted_data[pivoted_data['PENDS'] > '2011-03-31']
pivoted_data = pivoted_data[pivoted_data['PENDS'] < '2020-01-01']
# pivoted_data = pivoted_data[pivoted_data['PENDS'] > '2017-01-01']


pivoted_data

,OFTIC,PENDS,BPS,Brent_Close,Brent_Close_quarterly_return,CPS,CPX,CSH,C_Discretionary_Close,C_Discretionary_Close_quarterly_return,...,commodity_trade_Close_quarterly_return,industrials_Close,industrials_Close_quarterly_return,information_Close,information_Close_quarterly_return,materials_Close,materials_Close_quarterly_return,numeric_sector,utilities_Close,utilities_Close_quarterly_return
0,AAPL,2011-06-30,2.6718,112.48,-4.158146,0.422900,777.0,0.209015,40.209999,2.970550,...,0.062957,37.240002,-1.141483,25.700001,-1.381423,39.369999,-1.624192,3.0,33.480000,5.051769
1,AAPL,2011-09-30,2.9872,102.76,-8.641536,0.397600,1645.0,0.258659,34.860001,-13.305144,...,0.120922,29.219999,-21.535988,23.600000,-8.171207,29.360001,-25.425447,3.0,33.619999,0.418158
2,AAPL,2011-12-31,3.4500,107.38,4.495913,0.665700,1321.0,0.249095,39.020000,11.933447,...,-0.087046,33.750000,15.503083,25.450001,7.838985,33.500000,14.100815,3.0,35.980000,7.019633
3,AAPL,2012-03-31,3.9154,122.88,14.434718,0.528200,1457.0,0.153558,45.090000,15.556124,...,-0.191850,37.419998,10.874069,30.160000,18.506872,36.970001,10.358213,3.0,35.040001,-2.612559
4,AAPL,2012-06-30,4.2582,97.80,-20.410156,0.384300,2056.0,0.204900,43.779999,-2.905304,...,0.484614,35.669998,-4.676644,28.730000,-4.741380,35.290001,-4.544226,3.0,36.990002,5.565071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,ZBRA,2018-12-31,24.7813,53.80,-34.961315,5.329631,16.0,0.580469,99.010002,-15.534891,...,0.355782,64.410004,-17.844385,61.980000,-17.722026,50.520000,-12.791300,3.0,52.919998,0.512814
13481,ZBRA,2019-03-31,27.1789,68.39,27.118959,2.147867,15.0,0.279879,113.849998,14.988381,...,-0.178828,75.029999,16.488114,74.000000,19.393354,55.500000,9.857481,3.0,58.169998,9.920635
13482,ZBRA,2019-06-30,28.8100,66.55,-2.690452,3.577683,15.0,0.514700,119.199997,4.699164,...,0.081598,77.419998,3.185392,78.040001,5.459461,58.500000,5.405405,3.0,59.630001,2.509890
13483,ZBRA,2019-09-30,31.3430,60.78,-8.670173,2.272689,14.0,0.639219,120.699997,1.258389,...,-0.036101,77.629997,0.271247,80.529999,3.190669,58.200001,-0.512819,3.0,64.739998,8.569506


In [4]:
# pivoted_data.drop(columns=["Negative Prob", "Positive Prob", "Neutral Prob"], inplace=True)

In [5]:
# drop_ticker = {'BK',
#                 'CAH',
#                 'CPB',
#                 'CRM',
#                 'EIX',
#                 'EMR',
#                 'ETN',
#                 'ETR',
#                 'EXPD',
#                 'FAST',
#                 'FCX',
#                 'GIS',
#                 'GLW',
#                 'HIG',
#                 'HON',
#                 'IBM',
#                 'ILMN',
#                 'ITW',
#                 'KMB',
#                 'KR',
#                 'LLY',
#                 'MAS',
#                 'MCO',
#                 'MO',
#                 'MOH',
#                 'NRG',
#                 'NTRS',
#                 'ORLY',
#                 'OXY',
#                 'PM',
#                 'PNC',
#                 'PPL',
#                 'PSA',
#                 'RMD',
#                 'STT',
#                 'TXN',
#                 'VFC',
#                 'VMC',
#                 'XEL',
#                 'DFS',
#                 'ICE'}

# pivoted_data = pivoted_data[~pivoted_data['OFTIC'].isin(drop_ticker)]

In [6]:
print(list(pivoted_data.columns))

['OFTIC', 'PENDS', 'BPS', 'Brent_Close', 'Brent_Close_quarterly_return', 'CPS', 'CPX', 'CSH', 'C_Discretionary_Close', 'C_Discretionary_Close_quarterly_return', 'C_Staples_Close', 'C_Staples_Close_quarterly_return', 'DPS', 'EBG', 'EBI', 'EBS', 'EBT', 'ENT', 'EPS', 'EPS_forecast', 'Energy_Close', 'Energy_Close_quarterly_return', 'FFO', 'Financials_Close', 'Financials_Close_quarterly_return', 'GPS', 'GRM', 'Gold_Close', 'Gold_Close_quarterly_return', 'Health_care_Close', 'Health_care_Close_quarterly_return', 'Hrc_close', 'Hrc_close_quarterly_return', 'MEAN', 'NAV', 'NDT', 'NET', 'OPR', 'PRE', 'ROA', 'ROE', 'Real_Estate_Index_Price', 'Real_Estate_Index_Price_quarterly_return', 'SAL', 'STDEV', 'Three_Month_Yield', 'Three_Month_Yield_quarterly_return', 'VIX_Close', 'VIX_Close_quarterly_return', 'commodity_trade_Close', 'commodity_trade_Close_quarterly_return', 'industrials_Close', 'industrials_Close_quarterly_return', 'information_Close', 'information_Close_quarterly_return', 'materials_Clo

In [7]:
# unique_counts = pivoted_data.groupby('Sector')['OFTIC'].nunique()
# unique_counts

In [8]:
pivoted_data.columns

Index(['OFTIC', 'PENDS', 'BPS', 'Brent_Close', 'Brent_Close_quarterly_return',
       'CPS', 'CPX', 'CSH', 'C_Discretionary_Close',
       'C_Discretionary_Close_quarterly_return', 'C_Staples_Close',
       'C_Staples_Close_quarterly_return', 'DPS', 'EBG', 'EBI', 'EBS', 'EBT',
       'ENT', 'EPS', 'EPS_forecast', 'Energy_Close',
       'Energy_Close_quarterly_return', 'FFO', 'Financials_Close',
       'Financials_Close_quarterly_return', 'GPS', 'GRM', 'Gold_Close',
       'Gold_Close_quarterly_return', 'Health_care_Close',
       'Health_care_Close_quarterly_return', 'Hrc_close',
       'Hrc_close_quarterly_return', 'MEAN', 'NAV', 'NDT', 'NET', 'OPR', 'PRE',
       'ROA', 'ROE', 'Real_Estate_Index_Price',
       'Real_Estate_Index_Price_quarterly_return', 'SAL', 'STDEV',
       'Three_Month_Yield', 'Three_Month_Yield_quarterly_return', 'VIX_Close',
       'VIX_Close_quarterly_return', 'commodity_trade_Close',
       'commodity_trade_Close_quarterly_return', 'industrials_Close',
       

In [9]:
# pivoted_data.drop(columns=['Real_Estate_Index_Price','Three_Month_Yield', 'Brent_Close', 'Hrc_close',
#        'commodity_trade_Close', 'C_Discretionary_Close', 'C_Staples_Close',
#        'Energy_Close', 'Financials_Close', 'Health_care_Close',
#        'industrials_Close', 'information_Close', 'materials_Close',
#        'utilities_Close', 'numeric_sector',
#        'Real_Estate_Index_Price_quarterly_return',
#        'VIX_Close_quarterly_return', 'Gold_Close_quarterly_return',
#        'Three_Month_Yield_quarterly_return', 'Brent_Close_quarterly_return',
#        'Hrc_close_quarterly_return', 'commodity_trade_Close_quarterly_return',
#        'C_Discretionary_Close_quarterly_return',
#        'C_Staples_Close_quarterly_return', 'Energy_Close_quarterly_return',
#        'Financials_Close_quarterly_return',
#        'Health_care_Close_quarterly_return',
#        'industrials_Close_quarterly_return',
#        'information_Close_quarterly_return',
#        'materials_Close_quarterly_return', 'utilities_Close_quarterly_return'], inplace=True)

In [10]:
# pivoted_data = pivoted_data[pivoted_data["Sector"] == "Financials"]
# pivoted_data

In [11]:
unique_pends_by_ticker = pivoted_data.groupby('OFTIC')['PENDS'].nunique()
unique_pends_not_50 = unique_pends_by_ticker[unique_pends_by_ticker != 35]
unique_pends_not_50

Series([], Name: PENDS, dtype: int64)

In [12]:
pivoted_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9450 entries, 0 to 13484
Data columns (total 60 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   OFTIC                                     9450 non-null   object 
 1   PENDS                                     9450 non-null   object 
 2   BPS                                       9450 non-null   float64
 3   Brent_Close                               9450 non-null   float64
 4   Brent_Close_quarterly_return              9450 non-null   float64
 5   CPS                                       9450 non-null   float64
 6   CPX                                       9450 non-null   float64
 7   CSH                                       9450 non-null   float64
 8   C_Discretionary_Close                     9450 non-null   float64
 9   C_Discretionary_Close_quarterly_return    9450 non-null   float64
 10  C_Staples_Close                    

In [13]:
pivoted_data = pivoted_data.copy()

pivoted_data['Hrc_close'] = pd.to_numeric(pivoted_data['Hrc_close'].astype(str).str.split(",").str[0], errors='coerce')
pivoted_data['Gold_Close'] = pd.to_numeric(pivoted_data['Gold_Close'].astype(str).str.replace(",", ""), errors='coerce').round(2)

In [14]:
company_list = pivoted_data["OFTIC"].unique()
company_list

array(['AAPL', 'AFL', 'ALB', 'AMT', 'AMZN', 'ANSS', 'AON', 'AOS', 'APA',
       'APD', 'APH', 'ARE', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO',
       'BA', 'BDX', 'BEN', 'BG', 'BIIB', 'BK', 'BLK', 'BMY', 'BR', 'BSX',
       'BWA', 'BX', 'CAH', 'CBOE', 'CCI', 'CCL', 'CDNS', 'CE', 'CF',
       'CHD', 'CHRW', 'CHTR', 'CI', 'CMCSA', 'CME', 'CMI', 'CMS', 'CNC',
       'CNP', 'COF', 'COO', 'COST', 'CPB', 'CPRT', 'CPT', 'CRM', 'CSX',
       'CTSH', 'CVS', 'CVX', 'D', 'DAL', 'DE', 'DFS', 'DG', 'DGX', 'DHI',
       'DHR', 'DLR', 'DLTR', 'DOV', 'DPZ', 'DTE', 'DXCM', 'EBAY', 'ECL',
       'ED', 'EFX', 'EIX', 'EMR', 'EOG', 'EQIX', 'EQR', 'ESS', 'ETN',
       'ETR', 'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FAST', 'FCX',
       'FE', 'FFIV', 'FMC', 'FRT', 'FSLR', 'FTNT', 'GD', 'GILD', 'GIS',
       'GLW', 'GM', 'GNRC', 'GRMN', 'GS', 'GWW', 'HAL', 'HCA', 'HD',
       'HES', 'HIG', 'HOLX', 'HON', 'HPQ', 'HSY', 'HUM', 'IBM', 'ICE',
       'IDXX', 'IEX', 'INCY', 'IP', 'ISRG', 'IT', 'ITW', 'IVZ', 'JBHT

In [15]:
# pivoted_data = pivoted_data[["EPS","ROE","SAL","NET","EBI","EBT","GPS","DPS","BPS","NAV","OFTIC","PENDS","MEAN","STDEV", "BPS", "CPS", "PRE", "NDT"]]

In [16]:
len(company_list)

270

In [17]:
divisors = [i for i in range(1, len(company_list) + 1) if len(company_list) % i == 0]
divisors

[1, 2, 3, 5, 6, 9, 10, 15, 18, 27, 30, 45, 54, 90, 135, 270]

In [18]:
pivoted_data[pivoted_data["OFTIC"] == "ZTS"].sort_values(by="PENDS")

,OFTIC,PENDS,BPS,Brent_Close,Brent_Close_quarterly_return,CPS,CPX,CSH,C_Discretionary_Close,C_Discretionary_Close_quarterly_return,...,commodity_trade_Close_quarterly_return,industrials_Close,industrials_Close_quarterly_return,information_Close,information_Close_quarterly_return,materials_Close,materials_Close_quarterly_return,numeric_sector,utilities_Close,utilities_Close_quarterly_return


In [19]:
# pivoted_data.reset_index(inplace=True, drop=True)
# pivoted_data

In [20]:
stock_symbol = "PGR"
stock = pivoted_data[pivoted_data["OFTIC"] == stock_symbol].sort_values(by="PENDS")

stock_features_dict = {}
for column in stock.columns:
    stock_features_dict[column] = stock[column]

trace = go.Scatter(x=stock_features_dict["PENDS"], y=stock_features_dict["EPS"], mode="lines+markers", name=f"{stock_symbol}: EPS - PENDS")

layout = go.Layout(
    title = f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=600,
)

fig = go.Figure(data=trace, layout=layout)
pyo.iplot(fig)

In [21]:
pivoted_data[pivoted_data["OFTIC"]=="TROW"][["OFTIC","EPS","PENDS"]].sort_values(by=["OFTIC", "PENDS"])

,OFTIC,EPS,PENDS
12000,TROW,0.76,2011-06-30
12001,TROW,0.71,2011-09-30
12002,TROW,0.73,2011-12-31
12003,TROW,0.75,2012-03-31
12004,TROW,0.79,2012-06-30
12005,TROW,0.87,2012-09-30
12006,TROW,0.88,2012-12-31
12007,TROW,0.91,2013-03-31
12008,TROW,0.92,2013-06-30
12009,TROW,1.00,2013-09-30


In [22]:
pivoted_data[["EPS","OFTIC","PENDS"]].sort_values(by=["OFTIC", "PENDS"])

,EPS,OFTIC,PENDS
0,0.2782,AAPL,2011-06-30
1,0.2518,AAPL,2011-09-30
2,0.4954,AAPL,2011-12-31
3,0.4393,AAPL,2012-03-31
4,0.3329,AAPL,2012-06-30
...,...,...,...
13480,3.1000,ZBRA,2018-12-31
13481,2.9200,ZBRA,2019-03-31
13482,3.0200,ZBRA,2019-06-30
13483,3.4300,ZBRA,2019-09-30


In [23]:
X_ = pivoted_data.drop(columns=["EPS"])
y_ = pivoted_data['EPS']
y_

0        0.2782
1        0.2518
2        0.4954
3        0.4393
4        0.3329
          ...  
13480    3.1000
13481    2.9200
13482    3.0200
13483    3.4300
13484    3.5600
Name: EPS, Length: 9450, dtype: float64

In [24]:
company_dict = {}

for company in tqdm(pivoted_data["OFTIC"].unique()):
    comp_data = pivoted_data[pivoted_data["OFTIC"] == company].sort_values(by='PENDS')
    X = comp_data.drop(columns=["EPS","OFTIC", "PENDS"])
    # X = comp_data.drop(columns=["EPS", "Sector","OFTIC","Date", "PENDS"])
    y = comp_data['EPS']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    X_train_df = X_train
    X_test_df = X_test
    y_train_df = y_train
    y_test_df = y_test

    for column in X_train.columns:
        if "PENDS" in column:
            continue

        if "embedding" in column:
            continue
        
        if "Prob" in column:
            continue

        scaler = MinMaxScaler()

        X_train_scaled = scaler.fit_transform(X_train[[column]].values)
        X_train_df[column] = X_train_scaled
            
        X_test_scaled = scaler.transform(X_test[[column]].values)
        X_test_df[column] = X_test_scaled

    scaler_y = MinMaxScaler()

    y_train_scaled = pd.DataFrame(scaler_y.fit_transform(y_train.values.reshape(-1, 1)), columns=['EPS'], index=y_train.index)
    y_train_df = y_train_scaled
            
    y_test_scaled = pd.DataFrame(scaler_y.transform(y_test.values.reshape(-1, 1)), columns=['EPS'], index=y_test.index)
    y_test_df = y_test_scaled

    X_train_df["lagged_EPS"] = y_train_df
    X_test_df["lagged_EPS"] = y_test_df

    company_dict[company] = {"X_train": X_train_df, "X_test": X_test_df, "y_train": y_train_df, "y_test": y_test_df, "scaler_y": scaler_y}


  0%|          | 0/270 [00:00<?, ?it/s]

In [25]:
company_dict["PGR"]["X_test"]

,BPS,Brent_Close,Brent_Close_quarterly_return,CPS,CPX,CSH,C_Discretionary_Close,C_Discretionary_Close_quarterly_return,C_Staples_Close,C_Staples_Close_quarterly_return,...,industrials_Close,industrials_Close_quarterly_return,information_Close,information_Close_quarterly_return,materials_Close,materials_Close_quarterly_return,numeric_sector,utilities_Close,utilities_Close_quarterly_return,lagged_EPS
9678,1.138858,0.492523,0.808852,0.968741,0.732330,0.913012,1.120578,0.735004,0.803158,0.251749,...,0.913025,0.484770,1.096844,0.538344,0.921078,0.693463,0.0,0.944302,0.445998,0.871560
9679,1.312640,0.530841,0.671392,1.136925,0.738867,1.218710,1.239801,0.712070,0.891296,0.566433,...,1.058773,0.836612,1.236968,0.622477,0.916587,0.637155,0.0,0.979561,0.375589,1.036697
9680,0.998169,0.192991,0.069047,0.957238,1.365191,1.079371,0.965678,-0.077257,0.775615,0.076821,...,0.757589,0.099668,0.917743,-0.358002,0.678858,0.319639,0.0,0.993357,0.337380,0.954128
9681,1.360514,0.363435,1.025647,1.014460,0.950904,0.416721,1.189071,0.980329,0.971355,0.838735,...,0.986222,1.026594,1.205165,1.033229,0.838627,0.892645,0.0,1.261625,0.778369,1.036697
9682,1.610092,0.341939,0.566312,1.077870,1.792527,0.979763,1.269607,0.623823,1.043335,0.512132,...,1.037675,0.667441,1.301770,0.510931,0.934873,0.780009,0.0,1.336229,0.430992,1.119266
9683,1.770002,0.274533,0.474170,1.149039,0.986787,1.276714,1.292187,0.504605,1.166361,0.618266,...,1.042196,0.588763,1.361310,0.425888,0.925249,0.630280,0.0,1.597343,0.715035,1.036697
9684,1.677857,0.335514,0.740108,0.968847,1.056922,0.950983,1.363239,0.596497,1.223650,0.467675,...,1.124866,0.714989,1.627690,0.824818,1.028553,0.783228,0.0,1.591211,0.304654,0.899083


In [26]:
company_dict["TROW"]["X_train"]

,BPS,Brent_Close,Brent_Close_quarterly_return,CPS,CPX,CSH,C_Discretionary_Close,C_Discretionary_Close_quarterly_return,C_Staples_Close,C_Staples_Close_quarterly_return,...,industrials_Close,industrials_Close_quarterly_return,information_Close,information_Close_quarterly_return,materials_Close,materials_Close_quarterly_return,numeric_sector,utilities_Close,utilities_Close_quarterly_return,lagged_EPS
12000,0.032658,0.878505,0.543696,0.593023,0.208539,0.251689,0.080536,0.563929,0.057657,0.553415,...,0.172659,0.550621,0.050215,0.254508,0.321142,0.602163,0.0,0.000000,0.550142,0.047170
12001,0.000000,0.764953,0.474611,0.682171,0.198291,0.473431,0.000000,0.000000,0.000000,0.114769,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.007154,0.332943,0.000000
12002,0.049428,0.818925,0.677047,0.205426,0.465784,0.433615,0.062622,0.874480,0.103930,0.794207,...,0.097524,1.000000,0.044237,0.600125,0.132820,1.000000,0.0,0.127747,0.642385,0.018868
12003,0.115627,1.000000,0.830195,0.577519,0.000000,0.120564,0.153997,1.000000,0.162321,0.577455,...,0.176534,0.875023,0.156863,1.000000,0.244145,0.905314,0.0,0.079714,0.190879,0.037736
12004,0.051194,0.707009,0.293267,0.534884,0.139073,0.358470,0.134277,0.360339,0.187661,0.443646,...,0.138859,0.455177,0.122669,0.128563,0.190247,0.528287,0.0,0.179356,0.574203,0.075472
12005,0.195066,0.877453,0.837645,0.740310,0.163355,0.555681,0.179588,0.699222,0.226588,0.491400,...,0.157374,0.646533,0.172884,0.580276,0.238691,0.751507,0.0,0.148697,0.237308,0.150943
12006,0.179178,0.862500,0.590220,0.255814,0.209713,0.477393,0.189372,0.509137,0.192435,0.228173,...,0.186868,0.682694,0.125538,0.065555,0.262432,0.694129,0.0,0.073582,0.123988,0.160377
12007,0.256852,0.849766,0.592653,0.864341,0.245033,0.164767,0.273069,0.867086,0.371282,1.000000,...,0.269968,0.856411,0.159493,0.490786,0.315047,0.753780,0.0,0.287174,0.874442,0.188679
12008,0.322168,0.757944,0.497684,0.585271,0.154525,0.366330,0.324251,0.683277,0.367609,0.337496,...,0.288913,0.638333,0.167145,0.345915,0.288418,0.589659,0.0,0.212059,0.137113,0.198113
12009,0.395428,0.830491,0.701436,0.728682,0.163355,0.699661,0.387927,0.720867,0.372383,0.364506,...,0.370075,0.820146,0.201817,0.483967,0.405839,0.884706,0.0,0.198774,0.280954,0.273585


In [27]:
company_dict["PGR"]["y_test"].iloc[1]["EPS"]

1.0366972477064218

In [28]:
# Custom Dataset
class RollingWindowDataset(Dataset):
    def __init__(self, X, y, device=device):
        self.X = X.clone().detach().to(torch.float)
        self.y = y.clone().detach().to(torch.float)

    def __len__(self):
        return len(self.X) 

    def __getitem__(self, idx):
        # Ensure idx is within the valid range
        if idx > len(self.X):
            raise IndexError("Index out of bounds")

        X_window_normal = self.X[idx][0]
        X_window_years = self.X[idx][1]
        y_target = self.y[idx]  

        return X_window_normal.clone().detach().to(torch.float).to(device), X_window_years.clone().detach().to(torch.float).to(device), y_target.clone().detach().to(torch.float).to(device)

In [29]:
company_dict["TROW"]["X_train"]["lagged_EPS"]


12000    0.047170
12001    0.000000
12002    0.018868
12003    0.037736
12004    0.075472
12005    0.150943
12006    0.160377
12007    0.188679
12008    0.198113
12009    0.273585
12010    0.330189
12011    0.320755
12012    0.396226
12013    0.386792
12014    0.443396
12015    0.396226
12016    0.500000
12017    0.330189
12018    0.433962
12019    0.415094
12020    0.669811
12021    0.537736
12022    0.745283
12023    0.669811
12024    0.745283
12025    0.801887
12026    0.886792
12027    1.000000
Name: lagged_EPS, dtype: float64

In [30]:
# X_train = []
# y_train = []

# X_test = []
# y_test = []

# time_steps = 4
# print(len(company_dict[company]["X_train"]))
# print((len(company_dict[company]["X_test"])))

# for company in company_list:
#     company_dict[company]["y_train"]["Type"] = "Train"
#     company_dict[company]["y_test"]["Type"] = "Test"

#     comp_df_X = pd.concat([company_dict[company]['X_train'], company_dict[company]['X_test']], axis=0)
#     comp_df_y = pd.concat([company_dict[company]['y_train'], company_dict[company]['y_test']], axis=0)

#     for i in range((len(comp_df_X)) - time_steps):

#         if comp_df_y.iloc[i + time_steps]["Type"] == "Train":
#             X_train.append(comp_df_X.iloc[i : (i + time_steps)])
#             y_train.append(comp_df_y.iloc[i + time_steps]["EPS"])

#         elif comp_df_y.iloc[i + time_steps]["Type"] == "Test":
#             X_test.append(comp_df_X.iloc[i : (i + time_steps)])
#             y_test.append(comp_df_y.iloc[i + time_steps]["EPS"])


# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)

In [31]:
X_train = []
y_train = []

X_test = []
y_test = []

num_freq= 4
time_steps = 4
print(len(company_dict[company]["X_train"]))
print((len(company_dict[company]["X_test"])))

for company in company_list:
    company_dict[company]["y_train"]["Type"] = "Train"
    company_dict[company]["y_test"]["Type"] = "Test"

    comp_df_X = pd.concat([company_dict[company]['X_train'], company_dict[company]['X_test']], axis=0)
    comp_df_y = pd.concat([company_dict[company]['y_train'], company_dict[company]['y_test']], axis=0)

    for i in range((len(comp_df_X)) - time_steps):
        if i < 12:
            continue

        if comp_df_y.iloc[i + time_steps]["Type"] == "Train":

            years_df = pd.DataFrame()

            for freq in range(num_freq-1, -1, -1):
                start_idx = i + time_steps - (4 * (freq + 1))

                data_for_freq = comp_df_X.iloc[start_idx:start_idx+1, :]  
                years_df = pd.concat([years_df, data_for_freq], ignore_index=True, axis=0)

            # display(years_df)
            # display(comp_df_X.iloc[i : (i + time_steps)])

            normal_data = comp_df_X.iloc[i: (i + time_steps)].values
            years_data = years_df.values

            combined_sample = np.stack([normal_data, years_data], axis=0)
            
            X_train.append(combined_sample)
            y_train.append(comp_df_y.iloc[i + time_steps]["EPS"])

        elif comp_df_y.iloc[i + time_steps]["Type"] == "Test":

            years_df = pd.DataFrame()

            for freq in range(num_freq-1, -1, -1):
                start_idx = i + time_steps - (4 * (freq + 1))

                data_for_freq = comp_df_X.iloc[start_idx:start_idx+1, :]
                years_df = pd.concat([years_df, data_for_freq], ignore_index=True, axis=0)

            normal_data = comp_df_X.iloc[i: (i + time_steps)].values
            years_data = years_df.values

            combined_sample = np.stack([normal_data, years_data], axis=0)


            X_test.append(combined_sample)
            y_test.append(comp_df_y.iloc[i + time_steps]["EPS"])


X_train, y_train = np.array(X_train, dtype=np.float32), np.array(y_train, dtype=np.float32)
X_test, y_test = np.array(X_test, dtype=np.float32), np.array(y_test, dtype=np.float32)

28
7


In [32]:
X_train.shape

(3240, 2, 4, 58)

In [33]:
display(pd.DataFrame(y_train))

,0
0,0.292355
1,0.330512
2,0.788400
3,0.309699
4,0.143194
...,...
3235,0.433333
3236,0.500000
3237,0.671429
3238,0.890476


In [34]:
pd.DataFrame(X_train[0][0])

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.562419,0.877103,0.673546,0.106195,0.468403,0.316703,0.479904,0.569146,0.549394,0.518181,...,0.534769,0.670695,0.352702,0.512528,0.650626,0.769539,0.0,0.550843,0.629910,0.094630
1,0.492723,0.670444,0.365168,0.223451,0.892042,0.457601,0.479151,0.458408,0.567389,0.400438,...,0.515178,0.535993,0.389766,0.457789,0.649022,0.640706,0.0,0.439959,0.083523,0.143194
2,0.620045,0.234229,0.000000,1.000000,0.713868,0.530786,0.561343,0.744675,0.691517,0.698663,...,0.589021,0.755673,0.424438,0.442509,0.616619,0.591727,0.0,0.702095,0.884659,0.712085
3,0.692391,0.208294,0.548100,0.455752,0.465769,0.000000,0.609514,0.614677,0.700698,0.373267,...,0.571582,0.542789,0.426590,0.314448,0.623035,0.653670,0.0,0.559530,0.036382,0.458859


In [35]:
y_test[5]

0.70167893

In [36]:
X_train.shape

(3240, 2, 4, 58)

In [37]:
X_train.shape

(3240, 2, 4, 58)

In [38]:
y_test.shape

(1890,)

In [39]:
y_test

array([0.46232828, 0.6600527 , 1.1005967 , ..., 1.2190477 , 1.4142857 ,
       1.4761904 ], dtype=float32)

In [40]:
X_test.shape

(1890, 2, 4, 58)

In [41]:
X_train.shape

(3240, 2, 4, 58)

In [42]:
display(pd.DataFrame(X_train[0,0]))

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.562419,0.877103,0.673546,0.106195,0.468403,0.316703,0.479904,0.569146,0.549394,0.518181,...,0.534769,0.670695,0.352702,0.512528,0.650626,0.769539,0.0,0.550843,0.629910,0.094630
1,0.492723,0.670444,0.365168,0.223451,0.892042,0.457601,0.479151,0.458408,0.567389,0.400438,...,0.515178,0.535993,0.389766,0.457789,0.649022,0.640706,0.0,0.439959,0.083523,0.143194
2,0.620045,0.234229,0.000000,1.000000,0.713868,0.530786,0.561343,0.744675,0.691517,0.698663,...,0.589021,0.755673,0.424438,0.442509,0.616619,0.591727,0.0,0.702095,0.884659,0.712085
3,0.692391,0.208294,0.548100,0.455752,0.465769,0.000000,0.609514,0.614677,0.700698,0.373267,...,0.571582,0.542789,0.426590,0.314448,0.623035,0.653670,0.0,0.559530,0.036382,0.458859


In [43]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

train_data = RollingWindowDataset(X_train_tensor, y_train_tensor, device=device)
test_data = RollingWindowDataset(X_test_tensor, y_test_tensor, device=device)

print(test_data.__getitem__(0)[1].shape)
print(test_data.__getitem__(1)[0])


torch.Size([4, 58])
tensor([[0.9148, 0.2367, 0.9171, 0.4004, 0.9035, 1.0000, 0.8313, 0.4784, 0.8931,
         0.2677, 1.0000, 1.0000, 0.2511, 0.2742, 0.3213, 0.9125, 1.0000, 0.2397,
         0.6872, 1.0000, 0.8881, 0.6252, 0.3687, 0.0962, 0.3073, 0.6628, 0.9814,
         0.5388, 0.4444, 0.2943, 0.9412, 0.9132, 0.1532, 0.3043, 0.4217, 0.2810,
         0.0549, 0.3080, 0.4033, 0.7087, 0.4050, 0.6188, 0.6127, 0.2940, 0.0000,
         0.1488, 0.7656, 0.1143, 0.8995, 0.6960, 0.8489, 0.6063, 0.8803, 0.7838,
         0.0000, 1.0000, 0.4117, 0.3687],
        [1.0000, 0.3457, 0.8576, 0.9469, 0.5948, 0.5943, 0.9609, 0.7922, 1.0000,
         0.6006, 1.0000, 0.5320, 1.0000, 1.0000, 1.0000, 1.0000, 0.3545, 0.3306,
         0.6881, 0.5742, 1.0000, 0.6948, 1.0000, 0.1442, 0.3398, 0.6308, 1.0000,
         0.4632, 0.5791, 0.6245, 0.9386, 1.0000, 0.0000, 1.0000, 0.8406, 1.0000,
         0.3351, 0.9053, 0.4785, 0.7957, 1.0000, 0.7979, 0.8035, 0.3747, 0.0457,
         0.2997, 0.6857, 0.1207, 1.0000, 0.7590

In [44]:
# class VanillaLSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob):
#         super(VanillaLSTMModel, self).__init__()

#         self.hidden_size = hidden_size
#         self.layer_size = layer_size
#         self.hn_normal, self.cn_normal = None, None
#         self.hn_year, self.cn_year = None, None


#         self.lstm_normal = nn.ModuleList([
#             nn.LSTM(input_size=input_dim if i == 0 else hidden_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
#             for i in range(layer_size)
#         ])

#         self.lstm_year = nn.ModuleList([
#             nn.LSTM(input_size=input_dim if i == 0 else hidden_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
#             for i in range(layer_size)
#         ])

#         self.proj_normal = nn.ModuleList([
#             nn.Linear(input_dim, hidden_size) if i == 0 else nn.Identity()
#             for i in range(layer_size)
#         ])

#         self.proj_year = nn.ModuleList([
#             nn.Linear(input_dim, hidden_size) if i == 0 else nn.Identity()
#             for i in range(layer_size)
#         ])

#         self.norm_layers_normal = nn.ModuleList([
#             nn.LayerNorm(hidden_size) for _ in range(layer_size)
#         ])

#         self.norm_layers_year = nn.ModuleList([
#             nn.LayerNorm(hidden_size) for _ in range(layer_size)
#         ])

                            
#         self.dropout = nn.Dropout(dropout_prob)
        
#         self.fc = nn.Linear(self.hidden_size*2, output_dim)

#     def init_hidden(self, batch_size):
#         # Initialize hidden and cell states with zeros
#         h0 = torch.randn(1, batch_size, self.hidden_size).to(device)
#         c0 = torch.randn(1, batch_size, self.hidden_size).to(device)
#         return (h0, c0)
    
#     def reset_hidden(self):
#         self.hn_normal = None
#         self.cn_normal = None

#         self.hn_year = None
#         self.cn_year = None

#     def forward(self, data_normal, data_year):

#         if self.hn_normal == None or self.cn_normal == None:
#             self.hn_normal, self.cn_normal = self.init_hidden(data_normal.size(0))
#             self.hn_year, self.cn_year = self.init_hidden(data_year.size(0))
        
#         else:
#             self.hn_normal, self.cn_normal = self.hn_normal.detach(), self.cn_normal.detach()
#             last_hn_normal = self.hn_normal[:,-1:,:]
#             last_cn_normal = self.cn_normal[:,-1:,:]

#             self.hn_year, self.cn_year = self.hn_year.detach(), self.cn_year.detach()
#             last_hn_year = self.hn_year[:,-1:,:]
#             last_cn_year = self.cn_year[:,-1:,:]

#             self.hn_normal = last_hn_normal.repeat(1, data_normal.size(0), 1)
#             self.cn_normal = last_cn_normal.repeat(1, data_normal.size(0), 1)

#             self.hn_year = last_hn_year.repeat(1, data_year.size(0), 1)
#             self.cn_year = last_cn_year.repeat(1, data_year.size(0), 1)

#         out_normal = data_normal
#         out_year = data_year

#         projected_normal = data_normal
#         projected_year = data_year

#         # print(self.hn_normal.shape)
#         # print(self.hn_year.shape)
#         # Forward propagate LSTM
#         for i in range(self.layer_size):
#             out_normal, (self.hn_normal, self.cn_normal) = self.lstm_normal[i](out_normal, (self.hn_normal, self.cn_normal))
#             out_year, (self.hn_year, self.cn_year) = self.lstm_year[i](out_year, (self.hn_year, self.cn_year))

#             # projected_normal = self.proj_normal[i](projected_normal)
#             # projected_year = self.proj_year[i](projected_year)
        
#             # out_normal = out_normal + projected_normal
#             # out_year = out_year + projected_year

#             # out_normal = self.norm_layers_normal[i](out_normal)
#             # out_year = self.norm_layers_year[i](out_year)

#             # projected_normal = out_normal
#             # projected_year = out_year

#         out = torch.cat((out_normal, out_year), dim=2)

#         out = self.dropout(out[:, -1, :])  # Add dropout

#         out = self.fc(out)

#         return out

In [45]:
class VanillaLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob):
        super(VanillaLSTMModel, self).__init__()

        self.hidden_size = hidden_size
        self.layer_size = layer_size
        self.hn_normal, self.cn_normal = None, None
        self.hn_year, self.cn_year = None, None


        self.lstm_normal = nn.LSTM(input_size = input_dim, hidden_size = self.hidden_size, num_layers=self.layer_size,
                            dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
        self.lstm_year = nn.LSTM(input_size = input_dim, hidden_size = self.hidden_size, num_layers=self.layer_size,
                            dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
                            
        self.dropout = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(self.hidden_size*2, output_dim)

    def init_hidden(self, batch_size):
        # Initialize hidden and cell states with zeros
        h0 = torch.randn(self.layer_size, batch_size, self.hidden_size).to(device)
        c0 = torch.randn(self.layer_size, batch_size, self.hidden_size).to(device)
        return (h0, c0)
    
    def reset_hidden(self):
        self.hn_normal = None
        self.cn_normal = None

        self.hn_year = None
        self.cn_year = None

    def forward(self, data_normal, data_year):

        if self.hn_normal == None or self.cn_normal == None:
            self.hn_normal, self.cn_normal = self.init_hidden(data_normal.size(0))
            self.hn_year, self.cn_year = self.init_hidden(data_year.size(0))
        
        else:
            self.hn_normal, self.cn_normal = self.hn_normal.detach(), self.cn_normal.detach()
            last_hn_normal = self.hn_normal[:,-1:,:]
            last_cn_normal = self.cn_normal[:,-1:,:]

            self.hn_year, self.cn_year = self.hn_year.detach(), self.cn_year.detach()
            last_hn_year = self.hn_year[:,-1:,:]
            last_cn_year = self.cn_year[:,-1:,:]

            self.hn_normal = last_hn_normal.repeat(1, data_normal.size(0), 1)
            self.cn_normal = last_cn_normal.repeat(1, data_normal.size(0), 1)


            self.hn_year = last_hn_year.repeat(1, data_year.size(0), 1)
            self.cn_year = last_cn_year.repeat(1, data_year.size(0), 1)

        # print(self.hn_normal.shape)
        # print(self.hn_year.shape)
        # Forward propagate LSTM
        out_normal, (self.hn_normal, self.cn_normal) = self.lstm_normal(data_normal, (self.hn_normal, self.cn_normal))
        out_years, (self.hn_year, self.cn_year) = self.lstm_year(data_year, (self.hn_year, self.cn_year))

        out = torch.cat((out_normal, out_years), dim=2)

        out = self.dropout(out[:, -1, :])  # Add dropout

        out = self.fc(out)

        return out

In [46]:
# class VanillaLSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob):
#         super(VanillaLSTMModel, self).__init__()

#         self.hidden_size = hidden_size
#         self.layer_size = layer_size
#         self.hn = nn.Parameter(torch.zeros(self.layer_size, 1, self.hidden_size))
#         self.cn = nn.Parameter(torch.zeros(self.layer_size, 1, self.hidden_size))

#         self.lstm = nn.LSTM(input_size = input_dim, hidden_size = self.hidden_size, num_layers=self.layer_size,
#                             dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
                            
#         self.dropout = nn.Dropout(dropout_prob)
        
#         self.fc = nn.Linear(self.hidden_size, output_dim)

#     def init_hidden(self, batch_size):
#         # Initialize hidden and cell states with zeros
#         h0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         c0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         return (h0, c0)
    
#     def reset_hidden(self):
#         self.hn = None
#         self.cn = None

#     def forward(self, x):
#         hn = self.hn.repeat(1, x.size(0), 1)
#         cn = self.cn.repeat(1, x.size(0), 1)
        
#         # Forward propagate LSTM
#         out, (hn, cn) = self.lstm(x, (hn, cn))

#         out = self.dropout(out[:, -1, :])  # Add dropout

#         out = self.fc(out)

#         return out

In [47]:
# class VanillaLSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_size, layer_size, output_dim, dropout_prob):
#         super(VanillaLSTMModel, self).__init__()

#         self.hidden_size = hidden_size
#         self.layer_size = layer_size
#         self.hn, self.cn = None, None

#         self.lstm = nn.LSTM(input_size = input_dim, hidden_size = self.hidden_size, num_layers=self.layer_size,
#                             dropout=(dropout_prob if self.layer_size > 1 else 0), batch_first=True)
                            
#         self.dropout = nn.Dropout(dropout_prob)
        
#         self.fc = nn.Linear(self.hidden_size, output_dim)

#     def init_hidden(self, batch_size):
#         # Initialize hidden and cell states with zeros
#         h0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         c0 = torch.zeros(self.layer_size, batch_size, self.hidden_size).to(device)
#         return (h0, c0)
    
#     def reset_hidden(self):
#         self.hn = None
#         self.cn = None

#     def forward(self, x):
#         # self.hn, self.cn = self.init_hidden(x.size(0))
#         if self.hn == None or self.cn == None:
#             self.hn, self.cn = self.init_hidden(1)
#         else:
#             self.hn, self.cn = self.hn.detach(), self.cn.detach()

#         x_longer = x.view(1,x.shape[0]*x.shape[1],x.shape[2])

#         out_longer, (self.hn, self.cn) = self.lstm(x_longer, (self.hn, self.cn))

#         out = out_longer.view(x.shape[0],x.shape[1],out_longer.shape[2])

#         out = self.dropout(out[:,-1,:])
#         out = self.fc(out)

#         return out


In [48]:
def find_supported_splits(comp_size):
    supported_splits = []
    for n_splits in range(1, comp_size + 1):
        if comp_size % n_splits != 0:
            continue
        
        supported_splits.append(n_splits)
    return supported_splits


def custom_time_series_folds(data, n_splits):

    total_size = len(data)
    comp_size = total_size // len(company_list)
    comp_fold_size = comp_size//n_splits

    if comp_size % n_splits != 0:
        supported_splits = find_supported_splits(comp_size)
        print(supported_splits)
        print(f"fold_size: {comp_fold_size} comp_size: {comp_size}, company_list: {len(company_list)}, total_size: {total_size}")
        raise ValueError("Fold size must be divisible by the number of companies.")

    accumulated_train_idx = []     

    for i in range(n_splits-1):
        current_fold_val_idx = []
        current_fold_train_idx = []

        for j in range(len(company_list)):

            start_idx = j * comp_size
            val_start_idx = start_idx + (i+1) * comp_fold_size 
        
            end_idx = val_start_idx + comp_fold_size
        
            current_comp_train_idx = list(range(start_idx, val_start_idx))
            current_fold_train_idx.extend(current_comp_train_idx)  
        
            val_idx = list(range(val_start_idx, end_idx))
            current_fold_val_idx.extend(val_idx)  

        # print(f"train: {current_fold_train_idx}")
        # print(f"test: {current_fold_val_idx}")
        yield current_fold_train_idx, current_fold_val_idx

In [49]:
class ModelActioner:

    def __init__(self, train_data, test_data, device):
        self.train_data = train_data
        self.test_data = test_data
        self.device = device
        self.model = None
        self.optimizer = None
        self.criterion = nn.MSELoss()

    def custom_tscv(self, config, trial):
        batch_size = config["batch_size"]
        epochs = config["epochs"]
        hidden_size = config["hidden_size"]
        num_layers = config["layer_size"]
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = epochs//config["lr_step_size"]
        gamma = config["gamma"]

        suffle = False

        num_of_fold = 4

        fold_results = []

        for fold, (train_idx, val_idx) in enumerate(custom_time_series_folds(self.train_data, num_of_fold)):
            print(f"Fold: {fold+1}/{num_of_fold}")

            train_subset = Subset(self.train_data, train_idx)
            val_subset = Subset(self.train_data, val_idx)

            train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=suffle)
            val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=suffle)

            self.model = VanillaLSTMModel(input_dim=self.train_data.__getitem__(0)[0].shape[1], hidden_size=hidden_size, layer_size=num_layers, dropout_prob=dropout_prob, output_dim=1).to(self.device)

            self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
            scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min", verbose=True) 

            for epoch in range(epochs):
                print('epochs {}/{}'.format(epoch+1,epochs))

                running_loss = 0.0
                total_sample_train = 0
                
                self.model.reset_hidden()
                self.model.train()

                for batch_idx, (data_normal, data_year, target) in enumerate(train_loader):
                    target = target.view(-1,1) 

                    self.optimizer.zero_grad()

                    preds = self.model(data_normal, data_year)
                    loss = self.criterion(preds, target)
                    loss.backward()
                    self.optimizer.step() # Update model params

                    running_loss += loss.item() * data_normal.size(0)
                    total_sample_train += data_normal.size(0)

                train_loss = running_loss/total_sample_train

                self.model.eval()
                val_running_loss = 0.0
                total_sample_val = 0

                with torch.no_grad():

                    for batch_idx, (data_normal, data_year, target) in enumerate(val_loader):
                        target = target.view(-1,1)

                        preds = self.model(data_normal, data_year)
                        loss = self.criterion(preds, target)

                        val_running_loss += loss.item() * data_normal.size(0)
                        total_sample_val += data_normal.size(0)
                
                val_loss = val_running_loss/total_sample_val
                fold_results.append(val_loss)
                scheduler.step(val_loss)
                
                unique_step = fold * epochs + epoch
                trial.report(val_loss, unique_step)

                if trial.should_prune():
                    raise optuna.TrialPruned()

                current_lr = self.optimizer.param_groups[0]['lr']

                print(f'Current Learning Rate: {current_lr}')
                print(f"train_loss: {train_loss}, val_loss: {val_loss}")
                
        mean_val_loss = np.mean(fold_results)
        print(f"Mean validation loss: {mean_val_loss}")
        return mean_val_loss


                    
    def train(self, config):
        batch_size = config["batch_size"]
        epochs = config["epochs"]
        hidden_size = config["hidden_size"]
        num_layers = config["layer_size"]
        learning_rate = config["learning_rate"]
        dropout_prob = config["dropout_prob"]
        weight_decay = config["weight_decay"]
        lr_step_size = epochs//config["lr_step_size"]
        gamma = config["gamma"]

        self.model = VanillaLSTMModel(input_dim=self.train_data.__getitem__(0)[0].shape[1], hidden_size=hidden_size, layer_size=num_layers, dropout_prob=dropout_prob, output_dim=1).to(self.device)

        # Update optimizer with updated lr
        self.optimizer = optim.Adam(self.model.parameters(), lr = learning_rate, weight_decay=weight_decay)

        # Creating data loader
        train_loader = DataLoader(dataset=self.train_data, batch_size=batch_size, shuffle=False)

        scheduler = ReduceLROnPlateau(self.optimizer, patience=lr_step_size, factor=gamma, mode="min", verbose=True)  

        # Training Loop
        for epoch in range(epochs):
            print('epochs {}/{}'.format(epoch+1,epochs))
            
            running_loss = 0.0
            total_sample_train = 0

            self.model.reset_hidden()
            self.model.train()

            for batch_idx, (data_normal, data_year, target) in enumerate(train_loader):

                target = target.view(-1,1)  

                self.optimizer.zero_grad()
                preds = self.model(data_normal, data_year)

                loss = self.criterion(preds, target)
                loss.backward()
                self.optimizer.step() # Update model params

                running_loss += loss.item() * data_normal.size(0)
                total_sample_train += data_normal.size(0)

            train_loss = running_loss/total_sample_train
            scheduler.step(train_loss)
            current_lr = self.optimizer.param_groups[0]['lr']

            print(f'Current Learning Rate: {current_lr}')
            print(f"train_loss: {train_loss}")
        
        return self.model
            
    
    def test(self, config):
        batch_size = config["batch_size"]
        all_preds = []

        test_loader = DataLoader(dataset=self.test_data, batch_size=batch_size, shuffle=False)

        running_loss = .0
        total_sample = 0

        self.model.eval()

        with torch.no_grad():

            for batch_idx, (data_normal, data_year, target) in enumerate(test_loader):

                target = target.view(-1,1)
                
                preds = self.model(data_normal, data_year)
                loss = self.criterion(preds, target)

                running_loss += loss.item() * data_normal.size(0)
                total_sample += data_normal.size(0)

                all_preds.extend(preds.cpu().numpy())

            test_loss = running_loss/total_sample
            print(f"test_loss: {test_loss}")

        return all_preds
    


In [50]:
def objective(trial):
    config = {
        "batch_size": trial.suggest_int("batch_size", 22, 100),
        "epochs": trial.suggest_int("epochs", 100, 400, step=50),
        "hidden_size": trial.suggest_int("hidden_size", 200, 1000, step=10),
        "layer_size": trial.suggest_int("layer_size", 1, 6),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, step=0.0001),
        "dropout_prob": trial.suggest_float("dropout_prob", 0.1, 0.3, step=0.01),
        "weight_decay": trial.suggest_float("weight_decay", 1e-3, 1e-1, step=0.0001),
        "lr_step_size": trial.suggest_int("lr_step_size", 3, 10), 
        "gamma": trial.suggest_float("gamma", 1e-2, 1, step=0.01)
    }

    trainer = ModelActioner(train_data, test_data, device)

    val_loss = trainer.custom_tscv(config, trial)

    return val_loss

In [51]:
# study_name = "Vanilla-LSTM-Tunner"
# storage_url = "sqlite:///db.sqlite3"

# storage = optuna.storages.RDBStorage(url=storage_url)

# # Check if the study exists
# study_names = [study.study_name for study in optuna.study.get_all_study_summaries(storage=storage)]
# if study_name in study_names:
#     # Delete the study if it exists
#     print(f"Deleting study '{study_name}'")
#     optuna.delete_study(study_name=study_name, storage=storage_url)
# else:
#     print(f"Study '{study_name}' does not exist in the storage.")
    
# study = optuna.create_study(direction='minimize', 
#                             storage=storage_url, 
#                             sampler=TPESampler(),
#                             pruner=HyperbandPruner(
#                             min_resource=3,  # Minimum resource allocated to a trial
#                             max_resource='auto',  # Maximum resource allocated to a trial, 'auto' calculates it based on the first trial
#                             reduction_factor=2  # Reduction factor for pruning
#                             ),
#                             study_name=study_name,
#                             load_if_exists=False)

# pbar = tqdm(total=20, desc='Optimizing', unit='trial')

# def callback(study, trial):
#     # Update the progress bar
#     pbar.update(1)
#     pbar.set_postfix_str(f"Best Value: {study.best_value:.4f}")

# study.optimize(objective, n_trials=20, callbacks=[callback])
# pbar.close()

# # Best hyperparameters
# print('Number of finished trials:', len(study.trials))
# print('Best trial:')
# trial = study.best_trial

# print('Value:', trial.value)
# print('Params:')
# for key, value in trial.params.items():
#     print(f'{key}: {value}')

In [52]:
config = {
    "batch_size": 56,
    "epochs": 347,
    "hidden_size": 793,
    "learning_rate": 0.0014701180361360536,
    "dropout_prob": 0.16730859021906427,
    "weight_decay": 0.004094720505111237,
    "lr_step_size": 10, 
    "gamma": 0.07429231448874456,
    "layer_size": 1,
}

model = ModelActioner(train_data=train_data,test_data=test_data,device=device)
model.train(config)                

epochs 1/347
Current Learning Rate: 0.0014701180361360536
train_loss: 0.06402852694837399
epochs 2/347
Current Learning Rate: 0.0014701180361360536
train_loss: 0.045077491186007304
epochs 3/347
Current Learning Rate: 0.0014701180361360536
train_loss: 0.04765540283679226
epochs 4/347
Current Learning Rate: 0.0014701180361360536
train_loss: 0.04558000211592442
epochs 5/347
Current Learning Rate: 0.0014701180361360536
train_loss: 0.04366800188098425
epochs 6/347
Current Learning Rate: 0.0014701180361360536
train_loss: 0.0437300622785165
epochs 7/347
Current Learning Rate: 0.0014701180361360536
train_loss: 0.04407645928225032
epochs 8/347


In [ ]:
y_test = y_test.reshape(-1,1)
print(len(y_test))
y_test[:4]

In [ ]:
preds = model.test(config)

In [ ]:
preds = np.array(preds)
preds

In [ ]:
preds_inverse = []
y_true_inverse = []

idx = 0
data_size = len(y_test)//len(company_list)
print(data_size)

preds_comp_data = []
y_true_comp_data = pd.DataFrame()

for company in company_list:
    scaler = company_dict[company]["scaler_y"]
    preds_inverse.append(scaler.inverse_transform(preds[idx:idx+data_size]).flatten())
    y_true_inverse.append(scaler.inverse_transform(y_test[idx:idx+data_size]).flatten())

    for pred in preds_inverse:
        preds_comp_data.append({"Prediction": pred, "Company": company})

    idx += data_size



# for company in company_list:
#     preds_inverse.append(np.array(preds).flatten())
#     y_true_inverse.append(np.array(y_test).flatten())

comp_preds_inverse = np.array(preds_inverse)
comp_true_inverse = np.array(y_true_inverse)

preds_inverse = np.array(preds_inverse).flatten()
y_true_inverse = np.array(y_true_inverse).flatten()
print(y_true_inverse.shape)
print(preds_inverse.shape)



mse = mean_squared_error(y_true_inverse, preds_inverse)
mape = mean_absolute_percentage_error(y_true_inverse, preds_inverse)*100
mae = mean_absolute_error(y_true_inverse, preds_inverse)
r2 = r2_score(y_true_inverse,preds_inverse)

print(f"MAPE Score: %{mape:.2f}")
print(f"MSE Score: {mse:.2f}")
print(f"MAE Score: {mae:.2f}")
print(f"R_2 Score: {r2:.2f}")

In [ ]:
y_true_inverse

In [ ]:
preds_inverse = np.array(preds_inverse).flatten()
preds_inverse

In [ ]:
len(y_test)

In [ ]:
y_test.flatten()
x_indices = np.arange(len(y_true_inverse))
preds = np.array(preds)
preds

In [ ]:
len(y_true_inverse)

In [ ]:
trace_true = go.Scatter(x=x_indices, y=y_true_inverse, mode="lines", name=f"All Stocks True: EPS - PENDS")
trace_preds = go.Scatter(x=x_indices, y=preds_inverse, mode="lines", name=f"All Stocks Preds: EPS - PENDS")


layout = go.Layout(
    title = f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=600,
)

fig = go.Figure(data=[trace_true, trace_preds], layout=layout)
fig.show()

In [ ]:
x_indices = [["2018-Q2", "2018-Q3", "2018-Q4" ,"2019-Q1", '2019-Q2', '2019-Q3', '2019-Q4']] * len(company_list)

In [ ]:
stock_symbol = "Multiple Companies"

# Initialize the figure with layout
layout = go.Layout(
    title=f"{stock_symbol}: EPS - PENDS",
    xaxis=dict(title='Date'),
    yaxis=dict(title='EPS', side='left', rangemode='tozero'),
    height=700,
    width=1400
)
fig = go.Figure(layout=layout)

# Variables to keep track of the extended x-axis and y-axis data
extended_x_indices = []
all_true_y = []
all_preds_y = []

# Adjust x_indices and concatenate y data for each company
for i, company in enumerate(company_list):
    # Extend x_indices with company identifier to differentiate time points between companies
    extended_x_indices += [f"{x} {company}" for x in x_indices[i]]
    all_true_y += list(comp_true_inverse[i].flatten())
    all_preds_y += list(comp_preds_inverse[i].flatten())

# Add traces for the concatenated true and predicted EPS values
trace_true = go.Scatter(x=extended_x_indices, y=all_true_y, mode="lines+markers", name="All Companies True: EPS - PENDS")
trace_preds = go.Scatter(x=extended_x_indices, y=all_preds_y, mode="lines+markers", name="All Companies Preds: EPS - PENDS")

# Add traces to the figure
fig.add_trace(trace_true)
fig.add_trace(trace_preds)

# Show the figure
fig.show()

In [ ]:
pivoted_data[pivoted_data["OFTIC"]=="AAP"]